In [1]:
!pip list


Package                                  Version     Editable project location
---------------------------------------- ----------- -------------------------
absl-py                                  2.1.0
aiofiles                                 23.2.1
aiohttp                                  3.9.5
aiosignal                                1.3.1
altair                                   5.3.0
annotated-types                          0.6.0
anyascii                                 0.3.2
anyio                                    4.3.0
asgiref                                  3.8.1
async-timeout                            4.0.3
attrs                                    23.2.0
audioread                                3.0.1
Babel                                    2.14.0
backoff                                  2.2.1
bangla                                   0.0.2
bcrypt                                   4.1.2
blinker                                  1.7.0
blis                                    

In [2]:
!pip show langchain

Name: langchain
Version: 0.1.16
Summary: Building applications with LLMs through composability
Home-page: https://github.com/langchain-ai/langchain
Author: 
Author-email: 
License: MIT
Location: /home/bargh1/Teacher/lib/python3.10/site-packages
Requires: aiohttp, async-timeout, dataclasses-json, jsonpatch, langchain-community, langchain-core, langchain-text-splitters, langsmith, numpy, pydantic, PyYAML, requests, SQLAlchemy, tenacity
Required-by: 


In [3]:
!pwd

/home/bargh1/TTS


In [5]:
%cd ..

/home/bargh1/TTS


In [6]:
import sys
sys.path.append('/home/bargh1/Teacher/lib/python3.10/site-packages')

In [7]:
import langchain

In [8]:
!ls

 cc892f34-9370-4b86-9205-9db328c427b2	 pyproject.toml
 checkpoints				 README.md
 chroma.sqlite3				 recipes
 CITATION.cff				 requirements.dev.txt
 CODE_OF_CONDUCT.md			 requirements.ja.txt
 CODE_OWNERS.rst			 requirements.notebooks.txt
 CONTRIBUTING.md			 requirements.txt
 cross_dataset.ipynb			 run_bash_tests.sh
 Dockerfile				 S24-NLP-Syllabus-v1.pdf
 dockerfiles				 scripts
 docs					 setup.cfg
 flagged				 setup.py
 Hindi_checkpoints			'Teacher Assistant chatbot.ipynb'
 hubconf.py				 Teacher.ipynb
 images					 tests
 LICENSE.txt				 TTS
 LinearAlgebraReview-StanfordCS229.pdf	 TTS.egg-info
 Makefile				 wandb
 MANIFEST.in				 ZD_output
'Narges_Mohammadi1 (1).mp3'		'ZD_trainer (copy).py'
 notebooks				 ZD_trainer.ipynb
 Notes01-wordvecs1.pdf			 ZD_trainer.py
 Notes03-neuralnets.pdf


In [9]:
import tempfile
import soundfile as sf
from pydub import AudioSegment
from langchain_community.chat_models import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain
from langchain.chains import ChatVectorDBChain
from langchain.document_loaders import PyPDFLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.llms import OpenAI
import os
from TTS.tts.configs.xtts_config import XttsConfig
from TTS.tts.models.xtts import Xtts
import soundfile as sf  # Ensure this library is installed
import gradio as gr
import whisper
import gradio
from IPython.display import Audio
import tempfile

/home/bargh1/.local/lib/python3.10/site-packages/whisper/timing.py:58: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def backtrace(trace: np.ndarray):


In [10]:
import torch

In [11]:
chip = 0
device = torch.device(f"cuda:{chip}")

In [12]:
device

device(type='cuda', index=0)

In [13]:
whisper.available_models()
model1 = whisper.load_model("large-v2",device = device)
model1.device


device(type='cuda', index=0)

In [ ]:

def split_into_sentences(text, max_length=200):
    # Split the text into sentences based on punctuation marks
    sentences = []
    current_sentence = ""
    last_punctuation_index = 0  # To remember the last punctuation position

    for i, char in enumerate(text):
        current_sentence += char
        if char in ['.', '!', '?']:
            current_sentence = current_sentence[:-1] + ' ' + char
            last_punctuation_index = i  # Update the last known punctuation position

        if len(current_sentence) >= max_length:
            # If we reach max length, we should cut at the last punctuation
            if last_punctuation_index > 0:
                # Take up to the last punctuation
                sentences.append(current_sentence[:last_punctuation_index+1].strip())
                # Start new sentence after the last punctuation
                current_sentence = current_sentence[last_punctuation_index+1:].strip()
                last_punctuation_index = 0  # Reset punctuation index
            else:
                # No punctuation was found, but we need to split anyway
                sentences.append(current_sentence.strip())
                current_sentence = ""

    if current_sentence:
        sentences.append(current_sentence.strip())

    return sentences
def synthesize_speech(choice,text, speaker_wav="Narges_Mohammadi1 (1).mp3"):
    # Load the configuration
    config_path = "checkpoints/config.json"
    config = XttsConfig()
    config.load_json(config_path)

    # Initialize the model from the configuration
    model = Xtts.init_from_config(config)

    # Load the model checkpoint
    checkpoint_dir = "checkpoints"
    model.load_checkpoint(config, checkpoint_dir=checkpoint_dir, eval=True)

    # Move the model to GPU if you're using CUDA (uncomment the next line if needed)
    model.cuda()

    # Split the input text into smaller sentences
    sentences = split_into_sentences(text)

    # Create a list to store the paths of synthesized audio files
    audio_file_paths = []
    
    lang = set_lang(choice)
    print(lang)

    # Synthesize speech for each sentence
    for sentence in sentences:
        # Synthesize speech
        outputs = model.synthesize(
            sentence,
            config,
            speaker_wav=speaker_wav,
            gpt_cond_len=3,
            language=lang
        )

        # Extract the audio data
        audio_data = outputs['wav']
        samplerate = 22050  # Adjust based on your model's output sample rate

        # Save the audio data to a temporary file
        with tempfile.NamedTemporaryFile(suffix=".wav", delete=False) as temp_file:
            sf.write(temp_file.name, audio_data, samplerate)
            audio_file_paths.append(temp_file.name)

    # Concatenate the audio files
    combined_audio = AudioSegment.empty()
    for audio_file_path in audio_file_paths:
        audio_segment = AudioSegment.from_wav(audio_file_path)
        combined_audio += audio_segment

    # Save the combined audio to a temporary file
    with tempfile.NamedTemporaryFile(suffix=".wav", delete=False) as temp_file:
        combined_audio.export(temp_file.name, format="wav")
        combined_audio_path = temp_file.name

    # Return the path to the combined audio file
    return combined_audio_path
# Replace YOUR_OPENAI_API_KEY with your actual OpenAI API key
os.environ["OPENAI_API_KEY"] = "sk-proj-mqaHLOQf5awVkPM02F9OT3BlbkFJQAHciKYmKhIMkcUXtNHb"

def load_and_process_pdfs(pdf_paths):
    all_pages = []

    for pdf_path in pdf_paths:
        loader = PyPDFLoader(pdf_path)
        pages = loader.load_and_split()
        all_pages.extend(pages)  # Collect pages from all PDFs in a single list

    return all_pages

# Initialize embeddings
embeddings = OpenAIEmbeddings()

# Load multiple PDF files
pdf_paths = [
    'Notes01-wordvecs1.pdf',
    'Notes03-neuralnets.pdf',
    'S24-NLP-Syllabus-v1.pdf',
]
all_pages = load_and_process_pdfs(pdf_paths)

# Now use these pages to create embeddings and store them in a vector database
vectordb = Chroma.from_documents(all_pages, embedding=embeddings, persist_directory=".")

# Persist the vector database to disk
vectordb.persist()

print("Vector database created and persisted successfully.")

def transcribe(choice,audio_file_path):
    try:
        lang = set_lang(choice)
        # Load the audio file
        audio = whisper.load_audio(audio_file_path)

        # Transcribe the audio
        audio_to_text = model1.transcribe(audio, language="en", fp16=False)["text"]
        print("Transcribed Text:", audio_to_text)

        # Assuming vectordb is already initialized correctly
        pdf_ga = ChatVectorDBChain.from_llm(
            ChatOpenAI(temperature=1.2, model_name="gpt-4-turbo"),
            vectordb, return_source_documents=True
        )

        result = pdf_ga({"question": f"Please provide the answer in {choice}: {audio_to_text}", "chat_history": ""})
        result_in_farsi = result["answer"]
        audio_result = synthesize_speech(choice,result_in_farsi)

        return audio_to_text, result_in_farsi, audio_result

    except Exception as e:
        print("Error:", str(e))
        return "Error occurred during transcription", "", None
    
def set_lang(choice):
    if choice =="English":
        lang="en"
    if choice =="Hindi":
        lang="hi"
    if choice == "Persian":
        lang = "fa"
    return lang

choices=["English","Hindi","Persian"]
output_1 = gr.Textbox(label="Speech to Text (Whisper)")
output_2 = gr.Textbox(label="ChatGPT Output")
#output_3 = gr.Audio(label="ChatGPT output to audio via TTR", upload="output.wav")
output_3 = gr.Audio(label="ChatGPT output to audio via TTR")
gr.Interface(
    title='Persian Teacher assistant',
    fn=transcribe,
    inputs=[gr.Radio(choices),
        gr.Audio(source="microphone", type="filepath"),
    ],
    outputs=[output_1, output_2, output_3]
).launch(share=True, debug=True)

/home/bargh1/Teacher/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


Vector database created and persisted successfully.
Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://7fc91c42970013126e.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [ ]:
# import gradio as gr

# def set_lang(choice):
#     lang=choice
#     return f"You selected{lang}."

# choices=["English","Hindi","Persian"]
# radio = gr.Interface(
#     fn=set_lang,
#     inputs=gr.Radio(choices),
#     outputs=["text"],
#     title="Set Language",
# )

# radio.launch()

In [ ]:
# import gradio as gr

# def set_lang(English,Hindi,Persian):
#     if English:
#         lang = "Eng"
#     if Hindi:
#         lang = "Hi" 
#     if Persian:
#         lang = "Per" 
#     greeting = f"You have selected{lang}"
#     return greeting

# demo = gr.Interface(
#     set_lang,
#     inputs=["checkbox","checkbox", "checkbox"],
#     outputs=["text"],
# )
# if __name__ == "__main__":
#     demo.launch()

	